In [1]:
import nibabel as nib
import numpy as np
import pandas as pd
import gdist as gd
import ciftools_af as ct
import subprocess as sp
import statsmodels.api as sm
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy import stats


def vec2ang(x,y,degrees=False):
    ''' calculate angle of a vector given its x and y components'''
    vector_1 = [x, y]
    vector_2 = [np.abs(x), 0]

    unit_vector_1 = vector_1 / np.linalg.norm(vector_1)
    unit_vector_2 = vector_2 / np.linalg.norm(vector_2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.deg2rad(360) - np.arccos(dot_product) # to calculate angle anti-clockwise
    if degrees:
        angle = np.rad2deg(angle)
    return angle

/home/fralberti/.local/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
/home/fralberti/anaconda3/lib/python3.8/site-packages/statsmodels/compat/pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
### VARIABLES TO SET BEFORE RUNNING
# directory containing subdirectories named after subject IDs that contain the timeseries and surface files
root_dir = "/home/fralberti/Data/HCP/"
# directory where all intermediate files and the final output will be saved
output_dir = "/home/fralberti/Data/Output_misc/"
# list of IDs of subjects to include in the analyses
f = open(f'{root_dir}subj_IDs_200.txt', 'r')
subj_id = np.array(f.read().splitlines())
del f

In [ ]:
### GET GRADIENT PEAK
### Find the transmodal peak of the principal gradient in the lateral parietal area

peaks_df = pd.DataFrame(columns=['ID','hemisphere','peak','gradient_1'])
for subj in subj_id:
    grad = nib.load(f'{root_dir}{subj}/{subj}.gcca_200.32k_fs_LR.dscalar.nii')
    zones = nib.load(f'{root_dir}/HCP_S1200_GroupAvg_v1/zones.watershed.dlabel.nii')
    for hemi in ['L','R']:
        brain_structure = ['CIFTI_STRUCTURE_CORTEX_LEFT' if hemi=='L' else 'CIFTI_STRUCTURE_CORTEX_RIGHT'][0]
        offset, count, vertices = ct.struct_info(brain_structure, grad)
        # define a mask to limit peak search to the lateral parietal and occipital cortex
        z = [2 if hemi=='L' else 3][0]
        zone = zones.get_fdata()[20, offset : offset+count] == z
        # select vertices within the zone
        vtx_zone = vertices[zone]
        grad_zone = grad.get_fdata()[0, offset : offset+count][zone]
        # find vertex with the highest gradient value and its gradient score
        peak = vtx_zone[np.argmax(grad_zone)]
        grad_peak = grad_zone[np.argmax(grad_zone)]
        peaks_df =  peaks_df.append({'ID':subj, 'hemisphere':hemi, 'peak':peak,'gradient_1':grad_peak}, ignore_index=True)
    del grad
    
peaks_df.to_csv(f'{output_dir}grad1_lPAR_peaks.csv', index=False)

In [ ]:
### COMPUTE VECTOR LENGTH AND ANGLE
junction_df = pd.read_csv(f"{output_dir}gradientiles_200.csv")
junction_df = junction_df.loc[junction_df.ID_vtx==junction_df.ID_grad,['ID_vtx','hemisphere','vertex1','vertex2','vertex3']].reset_index(drop=True)
peak_df = pd.read_csv(f'{output_dir}grad1_zone2_peak.csv',index_col='ID')


vector_out = pd.DataFrame([],columns=['ID', 'hemisphere', 'Xv', 'Yv', 'distance', 'magnitude', 'angle'])
for subj in subj_id:
    for hemi in ['L','R']:
        # Find cetroid of the intersection triangle
        surface = nib.load(f'{root_dir}{subj}/Structural/{subj}.{hemi}.flat.32k_fs_LR.surf.gii')
        junct_vtx = junction_df.loc[(junction_df.ID_vtx==int(subj)) & (junction_df.hemisphere==hemi)]
        junct_vtx = np.array(junct_vtx.iloc[0,2:5], dtype='int32')
        junct_coord = np.array(surface.darrays[0].data)[junct_vtx,0:2]
        centroid_coord = junct_coord.mean(axis=0)
        
        # Get coordinates of the peak vertex
        subj_peaks = peak_df.loc[int(subj)]
        peak_vtx = np.array(subj_peaks.loc[subj_peaks.hemisphere==hemi , 'peak'], dtype='int32')
        peak_coord = np.array(surface.darrays[0].data)[peak_vtx,0:2]
        
        # Get geodesic distance
        vertices = np.float64(surface.darrays[0].data)
        triangles = np.int32(surface.darrays[1].data)
        dist = gd.compute_gdist(vertices, triangles, source_indices=junct_vtx, target_indices=peak_vtx)
        
        # Calculate angle
        v = (peak_coord - centroid_coord)[0]
        angle = vec2ang(v[0],v[1])
        
        # Get magnitude
        magtd = np.sqrt(np.power(v,2).sum())

        # Add to output
        vector_out = vector_out.append({'ID':subj, 'hemisphere':hemi, 'Xv':v[0], 'Yv':v[1],
                                        'distance':np.mean(dist), 'magnitude':magtd, 'angle':angle}, ignore_index=True)
vector_out.to_csv(f'{output_dir}/center_peak_vector.csv')